In [ ]:
from notebook_prelude import *

In [ ]:
import nltk
from nltk.corpus import reuters 
categories = reuters.categories()
c = collections.Counter()
multiple_indices= []
for idx, fileid in enumerate(reuters.fileids()):
    categories = reuters.categories(fileid)
    multiple = len(categories) > 1
    c['multiple' if multiple else 'one'] += 1
    if multiple:
        multiple_indices.append((idx, fileid))
#multiple_indices
sum(c.values())

In [ ]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
dataset = 'r8'
data = collections.defaultdict(lambda: [])
X, Y = dataset_helper.get_dataset(dataset)
# Add general dataset statistics
data['dataset'].append(dataset)
data['num_documents'].append(len(X))
data['num_classes'].append(len(set(Y)))

# Add text statistics
count_vec = CountVectorizer()
doc_vecs = count_vec.fit_transform(X)

all_words = set(count_vec.vocabulary_.keys())
data['num_words'].append(doc_vecs.sum())
data['num_unique_words'].append(len(all_words))
data['median_doc_length'].append(np.median([len(x) for x in X]))
data['median_words_per_doc'].append(np.median(np.squeeze(np.asarray(doc_vecs.sum(axis = 1)))))
pd.DataFrame(data)